In [1]:
import pandas as pd
import yfinance as yf
import psycopg2
from psycopg2 import sql
from sqlalchemy import create_engine

In [15]:
# Details for connection and api requests

# Database connection details
database = 'fin_data'
user = 'stevengiallourakis'
password = 'Birdman11!' # this might not be needed depenending on where you are connecting from.
host = 'localhost'
port =  '5433'
connection_string = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

# Details for api request

symbol = 'TGB'
start_date = '2020-01-01'
end_date = '2024-07-04'

In [25]:
# Get Stock Data
def get_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data = stock_data.drop(columns = ['High', 'Low', 'Adj Close', 'Volume'])
    stock_data['daily_change'] = stock_data['Close'] - stock_data['Open']
    stock_data['symbol'] = symbol
    stock_data.reset_index(inplace = True)
    print("Stock_data downloaded, High and Low columns removed, daily_change added")
    return stock_data 

# Add new data to database

def data_to_db(df, table_name, engine):
    try:
        df.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print("Data successfully added to the database.")
    except Exception as e:
        print(f"An error occurred while inserting data: {e}")
    

In [26]:
#Test

stock_data = get_stock_data(symbol, start_date, end_date)

data_to_db(stock_data, 'stocks', engine)

print(stock_data.head())

[*********************100%%**********************]  1 of 1 completed

Stock_data downloaded, High and Low columns removed, daily_change added
Data successfully added to the database.
        Date  Open  Close  daily_change symbol
0 2020-01-02  0.49   0.53          0.04    TGB
1 2020-01-03  0.54   0.52         -0.02    TGB
2 2020-01-06  0.52   0.52          0.00    TGB
3 2020-01-07  0.52   0.53          0.01    TGB
4 2020-01-08  0.54   0.54          0.00    TGB
